### Loading libraries

In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator as IDG
from keras.optimizers import Adam

Using TensorFlow backend.


# Convolution Neural network

In [2]:
split_ratio = 0.10
EPOCHS = 100
LR = 1e-3

In [3]:
t1 = time.time()

In [4]:
X = np.load('train_X_240_320.npy')
y = np.load('train_Y_60_80.npy')
print "input shape: {} \noutput shape: {}".format(X.shape, y.shape)

input shape: (1449, 240, 320, 3) 
output shape: (1449, 4800)


In [5]:
model = Sequential()

# First conv + activation + pooling layer
model.add(Conv2D(filters = 32, kernel_size=(5,5), strides=(1,1), input_shape = X[1].shape,data_format="channels_last"))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# second conv + activation + pooling layer
model.add(Conv2D(filters = 32,kernel_size= (5,5), strides=(1,1)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# third conv + activation + pooling layer
model.add(Conv2D(filters = 64,kernel_size= (5,5), strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Final decision layer 

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(y.shape[1]))


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 236, 316, 32)      2432      
_________________________________________________________________
dropout_1 (Dropout)          (None, 236, 316, 32)      0         
_________________________________________________________________
activation_1 (Activation)    (None, 236, 316, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 118, 158, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 114, 154, 32)      25632     
_________________________________________________________________
dropout_2 (Dropout)          (None, 114, 154, 32)      0         
_________________________________________________________________
activation_2 (Activation)    (None, 114, 154, 32)      0         
__________

#### Setting up tensoboard

In [7]:
adam = Adam(lr = LR, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)
model.compile(loss='mean_squared_error', optimizer = adam, metrics=['mae'])
tensorboard = TensorBoard(log_dir="./logs_2" , write_graph = True, write_images = True)

In [8]:
history = model.fit(X, y, epochs = EPOCHS, batch_size = 128,verbose = 1, validation_split = split_ratio 
                    , callbacks=[tensorboard], shuffle = True)

Train on 1304 samples, validate on 145 samples
Epoch 1/100
1304/1304 [==============================] - 1067s - loss: 0.8745 - mean_absolute_error: 0.5039 - val_loss: 0.0802 - val_mean_absolute_error: 0.2349
Epoch 2/100
1304/1304 [==============================] - 1127s - loss: 0.0310 - mean_absolute_error: 0.1330 - val_loss: 0.0920 - val_mean_absolute_error: 0.2566
Epoch 3/100
1304/1304 [==============================] - 1099s - loss: 0.0240 - mean_absolute_error: 0.1146 - val_loss: 0.0790 - val_mean_absolute_error: 0.2333
Epoch 4/100
1304/1304 [==============================] - 1088s - loss: 0.0221 - mean_absolute_error: 0.1105 - val_loss: 0.0844 - val_mean_absolute_error: 0.2433
Epoch 5/100
1304/1304 [==============================] - 961s - loss: 0.0198 - mean_absolute_error: 0.1052 - val_loss: 0.0874 - val_mean_absolute_error: 0.2486
Epoch 6/100
1304/1304 [==============================] - 1018s - loss: 0.0206 - mean_absolute_error: 0.1058 - val_loss: 0.0753 - val_mean_absolute_er

In [9]:
t2 = time.time()
print "Run time {} hr".format((t2-t1)/(60*60))

Run time 24.4380483575 min


In [10]:
model.save_weights('cnn_regression_complete_image2.h5') 

In [11]:
prediction = model.predict(X, verbose = 1)
predicted_depth = prediction.reshape((y.shape[0], 60, 80))

1449/1449 [==============================] - 97s    


In [12]:
actual_depth = y.reshape((y.shape[0] , 60 , 80))

In [14]:
def plotting(i) :
    plt.figure(figsize=(10,4))
    plt.subplot(1,3,1)
    plt.title("RGB Image")
    plt.imshow(X[i], cmap = 'gray', interpolation = 'bicubic')
    plt.subplot(1,3,2)
    plt.title("Actual Depth Map")
    plt.imshow(actual_depth[i], cmap = 'gray', interpolation = 'bicubic')
    plt.subplot(1,3,3)
    plt.title("Predicted Depth Map")
    plt.imshow(predicted_depth[i], cmap = 'gray', interpolation = 'bicubic')
    plt.show()